In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import h5py
from mayavi import mlab
import os
# import pyvista as pv
# mlab.init_notebook()

from mpl_toolkits.mplot3d import Axes3D

DataPath = os.path.abspath("").replace("Summer-Sandbox23/ptpg", "NbodySimulation/gevolution-1.2/output/")
newtonPath = DataPath+"newton/"
grPath = DataPath+"gr/"


In [ ]:
# List files in output
newtonFiles = os.listdir(newtonPath)
grFiles = os.listdir(grPath)

print(newtonFiles)
print(grFiles)



In [ ]:
# Extract datasets with .h5 formats
def Extracth5Specifics(filename:str) -> np.ndarray:
    h5File = h5py.File(filename, "r")
    dataset = h5File["data"][()]
    h5File.close()
    return dataset

def Extracth5Data(abspath:str) -> np.ndarray or list[np.ndarray]:
    Files = os.listdir(abspath)
    h5Files = [abspath+name for name in Files if ".h5" in name]
    if len(h5Files) > 1:
        datasets = []
        for filename in h5Files:
            datasets.append(Extracth5Specifics(filename))
        return datasets
    else:
        return Extracth5Specifics(h5Files[0])

newtonCube = Extracth5Data(newtonPath)
grCube = Extracth5Data(grPath)
print(f"Dimension of newton cube: {np.shape(newtonCube)}")
print(f"Dimension of gr cube: {np.shape(grCube)}")


In [ ]:
# filename = DataPath+"testrun/lcdm_snap003_hij.h5"

# h5_file = h5py.File(filename, "r")

# dataset = h5_file["data"]

# data = dataset[()]

# print(np.shape(data))


In [ ]:
# print(data[:,:,:,0].min())

In [ ]:
# x, y = np.mgrid[-3:3:100j, -3:3:100j]
# z = np.sin(x*y)

# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(111, projection="3d")

# # ax.plot_surface(x,y,z, cmap="viridis")

# ax.plot_trisurf(x.flatten(), y.flatten(), z.flatten(), cmap="viridis")

# plt.show()

In [ ]:
# mlab.contour3d(data, colormap="viridis")
# mlab.show()


# fig = plt.figure(figsize=(20,8))

# ncols, nrows = 3,2
# x, y, z, = np.meshgrid(range(64), range(64), range(64))

# for i in range(1,ncols*nrows+1):
#         ax = fig.add_subplot(nrows, ncols ,i, projection="3d")
#         im = ax.scatter(x.flatten(), y.flatten(), z.flatten(), c=data[:,:,:,i-1].flatten(), cmap="viridis")
#         ax.set_title(f"{i}")
# cbar = fig.colorbar(im, ax=fig.get_axes(), orientation="vertical")


